In [ ]:
using GeoDataFrames
using DataFrames
using Plots
using Shapefile
using Random
using ArchGDAL
using GeoInterface
using XLSX

In [ ]:
###Shapefile municipalities

shapefile_path = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\MT_Municipios_2022\\MT_Municipios_2022.shp"
municipalities = Shapefile.Table(shapefile_path) |> DataFrame

# Highway shapefile
shapefile_path_highways = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\Rodovias\\SNV_202410A.shp"
highways = Shapefile.Table(shapefile_path_highways) |> DataFrame

In [ ]:
# Filtering sg_uf equals to MT
filtered_highways_MT = filter(row -> row.sg_uf == "MT", highways)

# Grouping by vl_br row
grouped_highways = groupby(filtered_highways_MT, :vl_br)

In [ ]:
#Combine geometries
function combine_geometries(geometries)

    return mapfoldl(
        x -> GeoInterface.convert(ArchGDAL, x), # map - apply to each element
        ArchGDAL.union, # reduce 2 elements to 1 element
        geometries # the vector to perform the operation on
    )
end

In [ ]:
# Applying function
combined_geometries = combine(grouped_highways, :geometry => combine_geometries => :geometry)

#Showing results
println(combined_geometries)

In [ ]:
# Caminho para o arquivo Excel
file_path = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Table 1.xlsx"

# Carregar a planilha específica como uma tabela
table = XLSX.readtable(file_path, "Planilha1")

# Converter a tabela em DataFrame
colonization = DataFrame(table)

In [ ]:
# Realizar a junção dos dados
Merged = innerjoin(municipalities, colonization, on = :NM_MUN => :City)

# Filtrar as linhas onde a coluna Year não é nula
colonization_cities = filter(row -> !ismissing(row.Year), Merged)

In [ ]:
# Gerar a paleta de cores
num_cities = nrow(colonization_cities)
colors_colonization = distinguishable_colors(num_cities)

num_roads = nrow(combined_geometries)
colors_roads = distinguishable_colors(num_roads)


In [ ]:
# Definir tamanho do gráfico
plot(size=(2000, 1600))

# Plotar os municípios
plot(municipalities.geometry, color=:white, linecolor=:black, label=false)


In [ ]:
# Plotar os projetos de colonização
for (i, geom) in enumerate(colonization_cities.geometry)
    plot!(geom, color=colors_colonization[i], label=colonization_cities.NM_MUN[i])
end


In [ ]:
# Plotar as rodovias com cores diferentes e adicionar legenda
for (i, geom) in enumerate(combined_geometries.geometry)
    plot!(geom, color=colors_roads[i], linewidth=4, label="BR-" * string(combined_geometries.vl_br[i]))
end

In [ ]:
# Ajustar a legenda
plot!(legendfontsize=5, legend=:outerbottom, legend_lines = 8, legend_columns=4,legend_spacing=0.2)

# Remover eixos e grades
plot!(framestyle=:none, grid=false)

# Ajustar títulos e rótulos dos eixos
xlabel!("")
ylabel!("")
title!("")

In [ ]:
# Salvar o gráfico
savefig("C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Graph-Colonization with Highways.png")